# Fruit & Vegetables Freshness Classification

## Import Libraries

In [1]:
import tensorflow as tf
import tensorflow_datasets as tfds
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

## Download Dataset from Google Drive

In [2]:
pip install gdown

In [3]:
!gdown --id 1ftK3lZTKsKTXFivFHJfjpPzYiXDvNID2

In [4]:
import os
import zipfile 

data_dir = './fruitveg'
os.mkdir(data_dir)

with zipfile.ZipFile('fruitveg.zip', 'r') as zip_file:
    zip_file.extractall(data_dir)

## <span style='background :LemonChiffon' > Prepare dataset </span>

### Set parameters and load dataset

In [5]:
from tensorflow.keras.utils import image_dataset_from_directory

batch_size = 32
img_height = 224
img_width = 224
img_size = (img_height, img_width)
img_shape = img_size + (3,)

# Edit according to local path for dataset
ds_path = r"fruitveg"

train_ds = image_dataset_from_directory(ds_path,
                                        validation_split = 0.2,
                                        subset = "training",
                                        seed = 123,
                                        image_size = img_size,
                                        batch_size = batch_size)

val_ds = image_dataset_from_directory(ds_path,
                                      validation_split = 0.2,
                                      subset = "validation",
                                      seed = 123,
                                      image_size = img_size,
                                      batch_size = batch_size)

In [6]:
class_names = train_ds.class_names
num_classes = len(class_names)

print(class_names)

### Sample images

In [7]:
plt.figure(figsize=(13, 13))
for images, labels in train_ds.take(1):
    for i in range(25):
        ax = plt.subplot(5, 5, i + 1)
        plt.imshow(images[i].numpy().astype("uint8"))
        plt.title(class_names[labels[i]] + ' ({})'.format(labels[i]))
        plt.axis("off")

### Create test set

In [8]:
val_batches = tf.data.experimental.cardinality(val_ds)
test_dataset = val_ds.take(val_batches // 2)
validation_dataset = val_ds.skip(val_batches // 2)

In [9]:
# Buffered prefetching
AUTOTUNE = tf.data.AUTOTUNE

train_dataset = train_ds.prefetch(buffer_size = AUTOTUNE)
validation_dataset = validation_dataset.prefetch(buffer_size = AUTOTUNE)
test_dataset = test_dataset.prefetch(buffer_size = AUTOTUNE)

## <span style='background :LemonChiffon' > Building the model </span>

### Using transfer learning
Run one of the three following cells to choose from the ResNet50, MobileNetV2, and InceptionV3 pre-trained models. 

* ResNet50

In [10]:
# Create ResNet50 base model
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input, decode_predictions

pre_trained_model = ResNet50(input_shape = img_shape,
                        include_top = False,
                        weights = 'imagenet')

# pre_trained_model.trainable = False

* MobileNetV2

In [ ]:
# Create MobileNetV2 base model 
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2, preprocess_input

pre_trained_model = MobileNetV2(input_shape = img_shape,
                        include_top = False,
                        weights = 'imagenet')

# pre_trained_model.trainable = False

* InceptionV3

In [ ]:
# Create InceptionV3 base model
from tensorflow.keras.applications.inception_v3 import InceptionV3, preprocess_input

pre_trained_model = InceptionV3(input_shape = img_shape,
                        include_top = False,
                        weights = 'imagenet')

# pre_trained_model.trainable = False

In [ ]:
# Base model architecture
pre_trained_model.summary()

### Fine tuning layers

In [11]:
print("Number of layers: ", len(pre_trained_model.layers))

In [12]:
pre_trained_model.trainable = True

# Fine-tune from this layer onwards
START_TRAIN = 170

# Freeze all the layers before 
for layer in pre_trained_model.layers[:START_TRAIN]:
    layer.trainable = False

### Preprocess input and add classification layer

In [13]:
# Averaging layer
global_average = tf.keras.layers.GlobalAveragePooling2D()

# Data augmentation 
augmentation = tf.keras.Sequential([
    tf.keras.layers.RandomFlip('horizontal_and_vertical'),
    tf.keras.layers.RandomRotation(0.2),
    tf.keras.layers.RandomTranslation(height_factor = (-0.2, 0.2),
                                      width_factor = (-0.2, 0.2),
                                      fill_mode="nearest",
                                      interpolation="bilinear"),
    tf.keras.layers.RandomContrast(0.2)  
])

# Add dense layer
prediction_layer = tf.keras.layers.Dense(num_classes, activation='softmax')

# Chain model 
inputs = tf.keras.Input(shape = img_shape)
x = augmentation(inputs) 
x = preprocess_input(x)
x = pre_trained_model(x, training=False)
x = global_average(x)
x = tf.keras.layers.Dropout(0.5)(x)
outputs = prediction_layer(x)
model = tf.keras.Model(inputs,outputs)

### Compile model

In [14]:
# Compile model
LR = 0.0001
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate = LR),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits = False),
              metrics=['accuracy'])

# Display model architecture
model.summary()

### Building model from scratch
Run the following cell to build model from scratch, for transfer learning run the above cells.

In [ ]:
# Preprocessing
data_augmentation = tf.keras.Sequential([ 
    tf.keras.layers.RandomFlip('horizontal_and_vertical'),
    tf.keras.layers.RandomRotation(0.2)
])
    
rescale = tf.keras.layers.Rescaling(1./255) 
    
# Convolutional neural network
cnn = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(128, (3,3), activation='relu', input_shape=img_shape),
    tf.keras.layers.MaxPooling2D(2, 2),

    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),

    tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),

    tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),

    tf.keras.layers.Flatten(),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(num_classes, activation='softmax')
])

# Chain model
inputs = tf.keras.Input(shape = img_shape)
x = data_augmentation(inputs) 
x = rescale(x)
outputs = cnn(x)
model = tf.keras.Model(inputs,outputs)
    
# Compile model
lr = 0.001
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate = lr),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits = False),
              metrics=['accuracy'])

# Display model architecture
model.summary()

## <span style='background :LemonChiffon' > Model training </span>

In [15]:
# Callback function
class myCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        if(logs.get('accuracy')>0.98):
            print("\nReached 98% accuracy so cancelling training!")
            self.model.stop_training = True
            
callbacks = myCallback()

In [16]:
EPOCH = 100

history = model.fit(
            train_dataset,
            validation_data = validation_dataset,
            epochs = EPOCH,
            callbacks=[callbacks])

### Training Evaluation

In [17]:
plt.figure(figsize=(10, 9))

# training and validation accuracy
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

plt.subplot(2, 1, 1)
plt.plot(acc, label='Training Accuracy')
plt.plot(val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.ylim([min(plt.ylim()),1])
plt.title('Accuracy', size=15, fontweight='bold')

# training and validation loss
loss = history.history['loss']
val_loss = history.history['val_loss']

plt.subplot(2, 1, 2)
plt.plot(loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.ylabel('Cross Entropy')
plt.xlabel('Epoch')
plt.ylim([0,1.0])
plt.title('Loss', size=15, fontweight='bold')
plt.show()

## <span style='background :LemonChiffon' > Predicting images </span>

In [18]:
# Upload image to be predicted
from ipywidgets import FileUpload

upload = FileUpload()
upload

In [29]:
# Display image
import io
from PIL import Image

for name, file_info in upload.value.items():
    pic = Image.open(io.BytesIO(file_info['content']))

pic

In [30]:
# Resize image
x = np.array(pic)
x = tf.image.resize(x, [img_height, img_width])
x = np.expand_dims(x, axis=0)

# Predict image class
pred_img = model.predict(x)
pred_idx = np.argmax(pred_img, axis=-1)

print("It's a {}!".format(class_names[int(pred_idx)]))
print(pred_idx)
print(pred_img)

### Confusion Matrix

In [31]:
from sklearn.metrics import confusion_matrix
import seaborn as sns

predicted = []  # predicted labels
true = []  # true labels

for image_batch, label_batch in test_dataset:  
    true.append(label_batch)
    prediction = model.predict(image_batch)
    predicted.append(np.argmax(prediction, axis=-1))

# convert labels into tensors
true_labels = tf.concat([item for item in true], axis=0)
predicted_labels = tf.concat([item for item in predicted], axis=0)

cf_matrix = confusion_matrix(true_labels, predicted_labels, normalize='true')

# plot confusion  matrix
plt.figure(figsize = (10,7))
sns.heatmap(cf_matrix, 
            annot=True)
plt.title('Confusion Matrix', size=15, fontweight='bold')
plt.show()

## <span style='background :LemonChiffon' > Save model </span>

In [32]:
model.save('fruitveg_updated.h5')

In [33]:
tf.saved_model.save(model, 'fruitveg_updated')